In [ ]:
import nltk
import numpy as np
import pandas as pd
from pandas import DataFrame
from keras.models import Model, Sequential, load_model
from keras.preprocessing.sequence import TimeseriesGenerator, pad_sequences
from keras.layers.recurrent import LSTM
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.core import Activation, Dropout, Dense, Flatten
from keras.layers import TimeDistributed, Bidirectional, InputLayer, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from keras.metrics import categorical_accuracy
from IPython.display import clear_output
from more_itertools import flatten, intersperse
import random
from batcher import batch_from_generator
from train_data import load_conll2003, create_conll_encoded_shifted_generator
from mappings import get_all_mappings
from gutenberg import gutenberg_training_data_generator, gutenberg_book_generator_from_website
from padder import pad
import os
import urllib.request

In [ ]:
DROPOUT = 0.2
TIME_SLICE_SIZE = 128
BATCH_SIZE = 128
SAMPLING_RATE = 1
PADDING = 0
UNKNOWN = 1
NUM_OF_UNITS = TIME_SLICE_SIZE * 4
WORDS_PER_BATCH = 100
EPOCHS=1
MODEL_SAVE_PATH = 'tc_model.h5'
LSTM_MODEL_SAVE_PATH = 'lstm_model.h5'

In [ ]:
mapping, reverse_mapping, lower_mapping, lower_reverse_mapping = get_all_mappings()

In [ ]:
def create_model(shape, classes):
    # First layer inputs must be 3D
    # with shape (samples, timesteps, features)
    model = Sequential()
    model.add(InputLayer(input_shape=shape))
    model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
    model.add(Bidirectional(LSTM(NUM_OF_UNITS, return_sequences=True, dropout=DROPOUT, recurrent_dropout=DROPOUT)))
    model.add(TimeDistributed(Dense(classes)))
    model.add(Activation('softmax'))
    return model

In [ ]:
model = create_model((TIME_SLICE_SIZE, len(mapping)), len(mapping))

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
if os.path.isfile(MODEL_SAVE_PATH): 
    model.load_weights(MODEL_SAVE_PATH)

In [ ]:
tensor_board = TensorBoard(batch_size=BATCH_SIZE, write_graph=False, )

In [ ]:
printed_times = 0
g = gutenberg_training_data_generator(TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
print('training')
for i in range(0, 1000):
    X = None
    Y = None
    try:
        X, Y = next(g)
    except StopIteration:
        g = gutenberg_training_data_generator(TIME_SLICE_SIZE, BATCH_SIZE, shift=False)
        X, Y = next(g)
    if printed_times > 10:
        clear_output()
        print(f'epoch: {i}')
        printed_times = 0

    model.fit(X, Y,  verbose=2, validation_split=0.2, callbacks=[tensor_board])
    model.save(LSTM_MODEL_SAVE_PATH)
    printed_times += 1
print('done!')

In [ ]:
test_sentence = pad("Tim Hawkins born in 1989 in Birkenhead".lower(), TIME_SLICE_SIZE)
mapped_sentence = [lower_mapping[char] for char in test_sentence]
mapped_sentence = pad_sequences([mapped_sentence], maxlen=TIME_SLICE_SIZE, padding='post')
mapped_sentence = to_categorical(mapped_sentence, len(mapping))

mapped_sentence = np.asarray(mapped_sentence)
#mapped_sentence = np.reshape(1, len(test_sentence))
predicted_result = model.predict_classes(mapped_sentence)
predicted_result = [reverse_mapping[i] for i in predicted_result[0]]
''.join(predicted_result) 